In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
prod = ['140A',
        '143A',
        '349A',
        '582A',
        '964A',]

In [3]:
test = ['029A',
        '980A',
        '179A',
        '456A',
        '379A',
       ]

In [36]:
from collections import deque

# Numeric keypad layout
NUM_POS = {
    '7': (0,0), '8':(1,0), '9':(2,0),
    '4': (0,1), '5':(1,1), '6':(2,1),
    '1': (0,2), '2':(1,2), '3':(2,2),
    '0': (1,3), 'A':(2,3),
}
# Directional keypad layout
DIR_POS = {
    '^': (1,0), 'A':(2,0),
    '<': (0,1), 'v':(1,1), '>' :(2,1),
}


def check_grouped_button(path):
    if path == '':
        return True
    for i,ic in enumerate(path[:-1]):
        if ic != path[i+1] and path[i+1] in path[:i]:
            return False
    return True
    
def build_distances(pos_map):
    dists = {}
    for src in pos_map:
        d = {}
        dist_steps ={}
        sx, sy = pos_map[src]
        queue = deque([(sx, sy, "", 0)])
        seen = {(sx, sy): 0}
        while queue:
            x, y, path,steps = queue.popleft()
            
            if (x,y) in pos_map.values():
                tgt = next(k for k,v in pos_map.items() if v==(x,y))
                if check_grouped_button(path):
                    if tgt not in d.keys() or dist_steps[tgt] == steps:
                        dist_steps[tgt] = steps
                        try:
                            d[tgt].append(path)
                        except KeyError:
                            d[tgt] = [path]
                    
            for move, (dx, dy) in {'<':(-1,0),'>':(1,0),'^':(0,-1),'v':(0,1)}.items():
                nx, ny = x+dx, y+dy
                new_steps = steps + 1
                if (nx, ny) in pos_map.values() and ( (nx,ny) not in seen.keys() or new_steps <= seen[(nx,ny)] ):
                    seen[(nx,ny)] = new_steps
                    queue.append((nx, ny, path + move, new_steps))
        dists[src] = d
    return dists

# Compute movement maps for both keypads
NUM_PATHS = build_distances(NUM_POS)
DIR_PATHS = build_distances(DIR_POS)

# Compute Calculate Buttons necessary before
def generate_generating_sequence(code, shortest_paths):
    full_path = 'A'+code
    upper_code = ''
    for s,e in zip(full_path[:-1],full_path[1:]) :
        upper_code = upper_code+shortest_paths[s][e]+'A'
    return upper_code




In [37]:
def generate_generating_sequence(codes, shortest_paths):
    upper_return=[]
    for code in codes:
        full_path = 'A' + code
        upper_candidates = ['']
        for s, e in zip(full_path[:-1], full_path[1:]):
            candidates = shortest_paths[s][e]
            next_candidates = []
            for u in upper_candidates:
                for c in candidates:
                    next_candidates.append(u+c+'A')
            upper_candidates = next_candidates.copy()
    return upper_candidates

In [43]:
def part_one_complexity(codes):
    complexity=0
    for code in codes:
        r1 = generate_generating_sequence([code], NUM_PATHS)
        r2 =  generate_generating_sequence(r1, DIR_PATHS)
        person =  generate_generating_sequence(r2, DIR_PATHS)

        shortest_dist = min([len(p) for p in person])

        print(shortest_dist,int(code.strip('A')))
        
        complexity += shortest_dist*int(code.strip('A'))
    print(person)
    return complexity

In [44]:
%%time
part_one_complexity(test)

68 29
60 980
68 179
64 456
68 379
['v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>Av<A<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>Av<A<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA^>AA<A>A<vA<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA^>AA<A>A<vA<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA^>AA<A>Av<A<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA^>AA<A>Av<A<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>Av<A>^AA<A>A<vA<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>Av<A>^AA<A>A<vA<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>Av<A>^AA<A>Av<A<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>Av<A>^AA<A>Av<A<A>>^AAA<Av>A^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>Av<A^>AA<A>A<vA<A>>^AAA<A>vA^A', 'v<<A>>^AvA^Av<<A>>^AA<vA<A>>

127900

In [102]:
'v<<A>>^AvA^Av<<A>>^AAv   <A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A'
'<v<A>>^AvA^Av< A<  AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'

'v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A'

'<v<A>>^AvA^Av< A<  AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'

In [13]:
'v<<A>>^AvA^Av<<A>>^AAv   <A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A'.count('A')
'<v<A>>^AvA^Av< A<  AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'.count('A')

28

28

In [67]:
%%time
part_one_complexity(prod)

70 140
72 143
76 349
72 582
72 964
CPU times: user 302 μs, sys: 12 μs, total: 314 μs
Wall time: 316 μs


157932

# Part 2

In [40]:
def simulate_robot_layer_output(code_sequence, keypad_layout, start='A'):
    # Build reverse map from position to key
    pos_to_key = {v: k for k, v in keypad_layout.items()}
    x, y = keypad_layout[start]
    output = ''
    for ch in code_sequence:
        if ch in {'<', '>', '^', 'v'}:
            dx, dy = {'<': (-1, 0), '>': (1, 0), '^': (0, -1), 'v': (0, 1)}[ch]
            nx, ny = x + dx, y + dy
            if (nx, ny) in pos_to_key:
                x, y = nx, ny
            else:
                raise ValueError(f"Invalid move {ch} from position {(x, y)}")
        elif ch == 'A':
            output += pos_to_key[(x, y)]
        else:
            raise ValueError(f"Invalid input character: {ch}")
    return output


In [42]:
simulate_robot_layer_output('<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A', DIR_POS)

'<A>Av<<AA>^AA>AvAA^A<vAAA>^A'

In [45]:
simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)

'<A>A<AAv<AA>>^AvAA^Av<AAA^>A'

In [59]:
start='<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'
print(len(start), start)
tmp = simulate_robot_layer_output(start, DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

64 <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
28 <A>Av<<AA>^AA>AvAA^A<vAAA>^A
14 ^A<<^^A>>AvvvA


'379A'

In [60]:
start='v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A'
print(len(start), start)
tmp = simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

68 v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A
28 <A>A<AAv<AA>>^AvAA^Av<AAA^>A
14 ^A^^<<A>>AvvvA


'379A'

In [ ]:
<A>A<AAv<AA>>^AvAA^Av<AAA^>A
<A>Av<<AA>^AA>AvAA^A<vAAA>^A